In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wan = user_secrets.get_secret("wandb_api")
wandb.login(key=wan)

In [ ]:
wandb.init(project="misalignment-correction", name="encoder-decoder attention")

In [ ]:
import nibabel as nib
import os
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths to MRI and CT directories
mr_dir = "/kaggle/input/transformed-train/train_transformed/mr"
ct_dir = "/kaggle/input/transformed-train/train_transformed/ct"

# Check if the directories exist
if not os.path.exists(mr_dir):
    raise FileNotFoundError(f"Directory not found: {mr_dir}")
if not os.path.exists(ct_dir):
    raise FileNotFoundError(f"Directory not found: {ct_dir}")

# Get the first MRI and CT file paths
mr_files = sorted([os.path.join(mr_dir, f) for f in os.listdir(mr_dir) if f.endswith(".nii")])
ct_files = sorted([os.path.join(ct_dir, f) for f in os.listdir(ct_dir) if f.endswith(".nii")])

# Check if there are any files in the directories
if not mr_files or not ct_files:
    raise FileNotFoundError("No .nii files found in the specified directories.")

# Load the MRI and CT images
mr_image = nib.load(mr_files[0]).get_fdata()
ct_image = nib.load(ct_files[0]).get_fdata()

# Print the shapes of the MRI and CT images
print("MRI Image Shape:", mr_image.shape)
print("CT Image Shape:", ct_image.shape)


In [ ]:
import nibabel as nib
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform
from scipy.stats import entropy
import shutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths to MRI, CT, and mask directories
mr_dir = "/kaggle/input/transformed-train/train_transformed/mr"
ct_dir = "/kaggle/input/transformed-train/train_transformed/ct"
mask_dir = "/kaggle/input/transformed-train/train_transformed/mask"

# Check if the directories exist
if not os.path.exists(mr_dir) or not os.path.exists(ct_dir) or not os.path.exists(mask_dir):
    raise FileNotFoundError("One or more directories not found.")

# Get the MRI, CT, and mask file paths
mr_files = sorted([os.path.join(mr_dir, f) for f in os.listdir(mr_dir) if f.endswith(".nii")])
ct_files = sorted([os.path.join(ct_dir, f) for f in os.listdir(ct_dir) if f.endswith(".nii")])
mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(".nii")])

# Ensure all lists have the same length
assert len(mr_files) == len(ct_files) == len(mask_files), "Mismatch in number of files."

# Function to show a plane of an image
def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.axis("off")
    if title:
        ax.set_title(title)

# Function to calculate entropy
def calculate_entropy(image):
    # Calculate entropy from the gray-level histogram
    hist, _ = np.histogram(image.flatten(), bins=256, range=(0, 256))
    return entropy(hist)

# Function for simplified 3DVIR registration
def register_3dvir(fixed_image, moving_image):
    # Calculate the center of mass for both images
    fixed_center = np.array(np.where(fixed_image > 0)).mean(axis=1)
    moving_center = np.array(np.where(moving_image > 0)).mean(axis=1)

    # Calculate the translation vector
    translation = fixed_center - moving_center

    # Create an affine transformation matrix
    affine_matrix = np.eye(4)
    affine_matrix[:3, 3] = translation

    # Apply the affine transformation
    registered_image = affine_transform(moving_image, affine_matrix[:3, :3], offset=affine_matrix[:3, 3])

    return registered_image

# Create a directory to store processed images
output_dir = '/kaggle/working/processed_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process all pairs but visualize only the first 10
for i, (mr_file, ct_file, mask_file) in enumerate(zip(mr_files, ct_files, mask_files)):
    # Load images
    mr_image = nib.load(mr_file).get_fdata()
    ct_image = nib.load(ct_file).get_fdata()
    mask_image = nib.load(mask_file).get_fdata()

    # Apply 3DVIR with MRI as fixed image and CT as moving image
    registered_image = register_3dvir(mr_image, ct_image)

    # Visualize registered image only for the first 10 pairs
    if i < 10:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        show_plane(ax, registered_image[:, :, registered_image.shape[2] // 2], title=f"Registered CT Image {i}")
        plt.show()

    # Calculate and print metrics
    fixed_entropy = calculate_entropy(mr_image)
    moving_entropy = calculate_entropy(ct_image)
    registered_entropy = calculate_entropy(registered_image)

    print(f"Pair {i}:")
    print(f" Fixed Image (MRI) Entropy: {fixed_entropy:.4f}")
    print(f" Moving Image (CT) Entropy: {moving_entropy:.4f}")
    print(f" Registered Image (CT) Entropy: {registered_entropy:.4f}")
    print(f" Entropy Difference: {abs(fixed_entropy - registered_entropy):.4f}")
    print()

    # Create a folder for each pair
    pair_dir = os.path.join(output_dir, f'pair_{i}')
    if not os.path.exists(pair_dir):
        os.makedirs(pair_dir)

    # Save registered image (CT)
    registered_nii = nib.Nifti1Image(registered_image, np.eye(4))
    nib.save(registered_nii, os.path.join(pair_dir, f'registered_ct_image_{i}.nii'))

    # Copy corresponding MRI image to the pair folder
    shutil.copy(mr_file, os.path.join(pair_dir, f'mr_image_{i}.nii'))

print("Processing complete. Images saved in:", output_dir)
